# Exercise
In this exercise you will learn how to do trajectory optimisation using the Approximate Inference COntrol algorithm ([AICO](https://www.icml.cc/Conferences/2009/papers/271.pdf)).
The AICO algoritm is designed to solve a `UnconstrainedTimeIndexedProblem`. Similarly to the `UnconstrainedEdnPoseProblem` it defines a cost minimisation problem. However, the cost is minimized across an uniformly discretized trajectory. There is `T` number of timesteps spaced uniformly with at time intervals `tau` apart (`tau` would often be called `dt`). The task maps get updated for each time step and the task stores a vector of scaling factors `rho`. You can create complex transitions by tweaking the scaling factors for each task at every timestep separately.

The basic setup is the same as for every other solver since most of the problem definition takes place in the XML:

In [ ]:
import pyexotica as exo
import numpy as np
exo.Setup.init_ros()

In [ ]:
solver = exo.Setup.load_solver('resources/example_aico_trajectory.xml')
problem = solver.get_problem()

# Visualisation
scene = problem.get_scene()
vis = exo.VisualizationMeshcat(scene, 'tcp://127.0.0.1:6000')
vis.delete()
vis.display_scene()
exo.jupyter_meshcat.show(vis.get_web_url())

The weights `rho` can be set separately for each time step. A common setup is to set the weights to zero when a certain task does not need to be active. Below is an example of the end-effector frame tracking being ignored for the first 0.8s of the trajectory. Then the end-effector tracks the target (figure eight) with high precision (`1e5`). The solver doesn't have to minimise any cost for the 0.8s of the trajectory so it is only minimising the control effort. This setup will produce a smooth transition towards the target location before starting the treacking.

In [ ]:
# Set the rho profile
for t in range(0, problem.T):
    if float(t)*problem.tau < 0.8:
        problem.set_rho('Frame', 0.0, t)
    else:
        problem.set_rho('Frame', 1e5, t)

# Solve and visualize
trajectory = solver.solve()
vis.display_trajectory(trajectory, 0.3)

print('Planning time: '+str(solver.get_planning_time())+'s')

 1. Edit the code above to change the timing of the when the tracking starts.
 1. Edit the code above to encrease/decrease the tracking accuracy.
 1. Set the initial trajectory of the problem to warm start the solver, e.g., ```problem.initial_trajectory = trajectory```. Observe how the planning time decreases.